# Integration with External Verilog
This guide is targeted towards users working with existing Verilog designs (as
opposed to Python/Magma circuits).

The approach relies on Magma's external Verilog integration features to
construct a Magma circuit representation that can then be used with a
`fault.Tester` object. This approach enables the use of most of fault's
features, except for logic that relies on descending into a design hierarchy
(e.g. peeking and poking sub-instance ports).

The first, simplest approach is to use `m.DefineFromVerilog` or
`m.DefineFromVerilogFile` to import a verilog desing into Magma.  

Here's an example using `m.DefineFromVerilog`

In [1]:
import magma as m
import logging
logging.basicConfig(level=logging.INFO)
import fault

foo = m.DefineFromVerilog("""\
module foo(input I, output O);
assign O = I;
endmodule
""", target_modules=["foo"])[0]
print(foo)

tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)
# Note we currently have to tell magma to use the "verilog" 
# backend when working with a top-level verilog file
tester.compile_and_run("verilator", magma_output="verilog")

INFO:root:Running tester...


foo(I: In(Bit), O: Out(Bit))


INFO:root:Success!


Here's an example using `m.DefineFromVerilogFile`

In [2]:
with open("foo.v", "w") as f:
    f.write("""\
module foo(input I, output O);
assign O = I;
endmodule
""")
    
foo = m.DefineFromVerilogFile("foo.v", target_modules=["foo"])[0]
print(foo)

tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)
tester.compile_and_run("verilator", magma_output="verilog")

INFO:root:Running tester...


foo(I: In(Bit), O: Out(Bit))


INFO:root:Success!


An alternative to using `DefineFromVerilog` is to use `DeclareFromVerilog` to import a module interface, and provide the implementation to the simulator by copying the source verilog file into the simulation directory.  This is useful when the source file contains code that is not supported by Magma's Verilog parser (e.g. advanced system verilog features), or when parsing takes a long time (e.g. a post-synthesis netlist file).

In [5]:
with open("foo_stub.v", "w") as f:
    f.write("""\
module foo(input I, output O);
endmodule
""")

# You can similarly use DeclareFromVerilog with a Verilog string
foo = m.DeclareFromVerilogFile("foo_stub.v", target_modules=["foo"])[0]
print(foo)

tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)

import tempfile
import shutil

with tempfile.TemporaryDirectory() as dir_:
    # Copy actual implementation to test directory
    shutil.copy("foo.v", dir_)
    # Set test directory with directory= kwarg
    tester.compile_and_run("verilator", directory=dir_, skip_compile=True)

INFO:root:Running tester...


foo(I: In(Bit), O: Out(Bit))


INFO:root:Success!


A similar approach is to declare the interface using magma. This has the advantage of providing the capability of writing a sophisticated interface generator for your external module (e.g. if you're integrating with an external generator framework).

In [7]:
class foo(m.Circuit):
    IO = ["I", m.In(m.Bit), "O", m.Out(m.Bit)]

tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)

import tempfile
import shutil

with tempfile.TemporaryDirectory() as dir_:
    # Copy actual implementation to test directory
    shutil.copy("foo.v", dir_)
    # Set test directory with directory= kwarg
    tester.compile_and_run("verilator", directory=dir_, skip_compile=True)

INFO:root:Running tester...
INFO:root:Success!
